In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object 

c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
f = open('test_train_dataset.json')

TRAIN_DATA = json.load(f)

In [4]:
TRAIN_DATA

{'classes': ['CELL', 'VALUE'],
 'annotations': [['Insert a1 20\r',
   {'entities': [[7, 9, 'CELL'], [10, 12, 'VALUE']]}],
  ['Insert 50 into b3\r', {'entities': [[7, 9, 'VALUE'], [15, 17, 'CELL']]}],
  ['Put Dhruv in c4', {'entities': [[4, 9, 'VALUE'], [13, 15, 'CELL']]}]]}

In [4]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start,end,label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./test_train_dataset.spacy")

100%|██████████| 3/3 [00:00<00:00, 3003.08it/s]


CPU config file

In [7]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


GPU Config file

In [5]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Train dataset and test dataset is same in this case

In [7]:
!python -m spacy train config.cfg --output ./ --paths.train ./test_train_dataset.spacy --paths.dev ./test_train_dataset.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00      9.70    0.00    0.00    0.00    0.00
200     200          0.03     60.84  100.00  100.00  100.00    1.00
⚠ Aborting and saving the final best model. Encountered exception:
MemoryError((2500, 256), dtype('float32'))


Traceback (most recent call last):
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 135, in train
    raise e
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 118, in train
    for batch, info, is_best_checkpoint in training_step_iterator:
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 236, in train_while_improving
    proc.finish_update(optimizer)  # type: ignore[attr-defined]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "spacy\pipeline\trainable_pipe.pyx", line 252, in spacy.pipeline.trainable_pipe.TrainablePipe.finish_update
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\thinc\model.py", line 346, in finish_update
    param, grad = optimizer(
                  ^^^^^^^^^^
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python

In [2]:
nlp_ner = spacy.load("model-best")

In [7]:
doc = nlp_ner("Inject 40 at B6")

from spacy import displacy

displacy.render(doc,style="ent")